In [1]:
# Ignore this stuff, only useful for Google Drive stuff
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/CamCAN

/content/drive/My Drive/CamCAN


In [3]:
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import os
from random import sample, choice
from keras.layers import Dense, Conv3D, Flatten, Dropout, Conv3DTranspose, LeakyReLU, Input, Concatenate, Subtract, BatchNormalization, ZeroPadding3D, Cropping3D, InputLayer
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential, load_model, Model
from sklearn.model_selection import train_test_split

In [4]:
# Loss functions. Mix is a combination of MS-SSIM loss (high contrast, good looking) and L1 loss (high accuracy)
# The concept of the mixed loss is described here: https://arxiv.org/pdf/1511.08861.pdf

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))


def MSSSIM(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim_multiscale(
    y_true, y_pred, 1, filter_size=6,
    filter_sigma=1.5, k1=0.01, k2=0.03))

def L1(y_true, y_pred):
  return tf.reduce_mean(K.abs(y_true-y_pred))

def Mix(y_true, y_pred):
  MSSSIM_loss = 1 - tf.image.ssim_multiscale(
    y_true, y_pred, 1, filter_size=6,
    filter_sigma=1.5, k1=0.01, k2=0.03)
  L1_loss = K.abs(y_true-y_pred)
  return tf.reduce_mean(0.7*MSSSIM_loss+ 0.3*L1_loss)


In [5]:
# Load the autoencoders (encoder/decoder models, see other notebooks for training)
# Totally unnecessary once we get off of Colaboratory and onto Mist or Vector computers

full_autoencoder_dti = load_model('models/7_28_autoencoder_dti.h5', custom_objects={'L1':L1})
full_autoencoder_t1 = load_model('models/7_28_autoencoder_t1.h5', custom_objects={'MSSSIM': MSSSIM})

encoder_dti = Sequential()
for i in range(2): #3 / 3,7
  encoder_dti.add(full_autoencoder_dti.layers[i])

encoder_t1 = Sequential()
for i in range(2):
  encoder_t1.add(full_autoencoder_t1.layers[i])

decoder_dti=Sequential()
decoder_dti.add(InputLayer((80, 100, 85, 1)))
for i in range(2, 4):
  decoder_dti.add(full_autoencoder_dti.layers[i])

decoder_t1=Sequential()
decoder_t1.add(InputLayer((80, 100, 85, 1)))
for i in range(2, 4):
  decoder_t1.add(full_autoencoder_t1.layers[i])


In [9]:
# Code for loading T1 and FA data.

dtis = 0
t1s = 0

actual_name_list = []
for individual in os.listdir('data/t1_coreg')[10:]:
    if os.path.exists('data/t1_coreg/' +individual +'/anat/'+individual +'_T1w_ants.nii.gz') and os.path.exists('data/dwi_coreg/' +individual +'/dwi/dti_FA_ants.nii.gz'):
        print(individual)
        try:
          loaded_T1 = nb.load('data/t1_coreg/' +individual +'/anat/'+individual +'_T1w_ants.nii.gz').get_fdata()
          loaded_FA = nb.load('data/dwi_coreg/' +individual +'/dwi/dti_FA_ants.nii.gz').get_fdata()

          # this slicing was originally also done as an image compression for computational purposes
          # not only will this be unnecessary, it also won't work on our new dataset since we're no longer pre-registering our images.
          loaded_T1 = loaded_T1[10:170, 10:210, 0:170] 
          encoded_loaded = encoder_t1.predict(np.expand_dims(np.expand_dims(loaded_T1, 0), -1)/1000)[0]

          if not t1s is 0:
            t1s = np.concatenate([np.expand_dims(encoded_loaded, axis=0), t1s], axis=0)
          else:
            t1s = np.expand_dims(encoded_loaded, axis=0)
          del loaded_T1

          # this slicing was originally also done as an image compression for computational purposes
          # not only will this be unnecessary, it also won't work on our new dataset since we're no longer pre-registering our images.
          loaded_FA = loaded_FA[10:170, 10:210, 0:170]
          encoded_loaded = encoder_dti.predict(np.expand_dims(np.expand_dims(loaded_FA, 0), -1))[0]

          if not dtis is 0:
            dtis = np.concatenate([np.expand_dims(encoded_loaded, axis=0), dtis], axis=0)
          else:
            dtis = np.expand_dims(encoded_loaded, axis=0)
          del loaded_FA
          actual_name_list.append(individual)
        except Exception:
          print('Error')
    else:
        print('Error in ' + individual)


sub-CC220419
sub-CC220806
sub-CC220535
sub-CC310256
sub-CC223085
sub-CC222797
sub-CC221886
sub-CC221002
sub-CC221054
sub-CC220920
sub-CC221755
sub-CC310052
sub-CC220843
sub-CC310450
sub-CC320202
sub-CC320359
sub-CC410129
Error in sub-CC322186
Error in sub-CC320621
Error in sub-CC320680
sub-CC320417
Error in sub-CC320574
Error in sub-CC321291
Error in sub-CC320888
sub-CC410040
sub-CC410182
sub-CC420100
sub-CC420566
Error in sub-CC420180
sub-CC510039
sub-CC420231
sub-CC410220
sub-CC420094
sub-CC420222
sub-CC420582
sub-CC510076
sub-CC510115
sub-CC510237
sub-CC510259
sub-CC510648
sub-CC510329
sub-CC520136
sub-CC510474
sub-CC510483
sub-CC510354
sub-CC510323
sub-CC520055
sub-CC520197
sub-CC520775
sub-CC620262
sub-CC520287
sub-CC520980
sub-CC520377
sub-CC520624
sub-CC610671
sub-CC520562
sub-CC610099
sub-CC620413
sub-CC620526
Error in sub-CC710486
sub-CC620499
sub-CC710131
Error
Error in sub-CC710518
Error in sub-CC710462
Error in sub-CC711128
sub-CC621011
Error in sub-CC710342
sub-CC711245
su

In [10]:
anat = t1s
print(anat.shape)

(64, 80, 100, 85, 1)


In [11]:
# Model code.
# 3D convolution with kernel size 2, filters 128, and relu activation x5
# Trained with Adam optimizer, our loss function is the mixed loss.

mse_generator = Sequential()
mse_generator.add(Conv3D(filters=128, kernel_size=2, activation='relu', padding='same', input_shape = (t1s.shape[1], t1s.shape[2], t1s.shape[3], 1)))
mse_generator.add(Conv3D(filters=128, kernel_size=2,  activation='relu', padding='same'))
mse_generator.add(Conv3D(filters=128, kernel_size=2,  activation='relu', padding='same'))
mse_generator.add(Conv3D(filters=128, kernel_size=2,  activation='relu', padding='same'))
mse_generator.add(Conv3D(filters=128, kernel_size=2,  activation='relu', padding='same'))
mse_generator.add(Conv3D(filters=1, kernel_size=2, padding='same', activation = 'relu'))

mse_generator.compile(optimizer='adam',loss=Mix)


In [12]:
train_t1s, test_t1s, train_dtis, test_dtis = train_test_split(anat, dtis, test_size=0.1)

In [14]:
for iteration in range(100):
  for _ in range(10):
    indices = sample(list(range(len(train_t1s))), 1)
    mse_generator.fit(train_t1s[indices], train_dtis[indices], epochs=1)# validation_data = (test_t1s, test_dtis),
  if iteration % 3 == 0:
    ref_image = np.expand_dims(train_dtis[0], axis=0)
    train_image = np.expand_dims(train_t1s[0], axis=0)
    test_image = np.expand_dims(test_t1s[0], axis=0)
    ref_test = np.expand_dims(test_dtis[0], axis=0)

    plt.imshow(decoder_dti.predict(ref_image)[0, 40, :, :, 0], cmap='Greys')
    plt.show()
    plt.imshow(decoder_dti.predict(mse_generator.predict(train_image))[0, 40, :, :, 0], cmap='Greys')
    plt.show()

    plt.imshow(decoder_dti.predict(ref_test)[0, 40, :, :, 0], cmap='Greys')
    plt.show()
    plt.imshow(decoder_dti.predict(mse_generator.predict(test_image))[0, 40, :, :, 0], cmap='Greys')
    plt.show()

    mse_generator.save('models/8_11_generator.h5')

Output hidden; open in https://colab.research.google.com to view.